In [ ]:
import os

: 

In [3]:
os.environ["GROQ_API_KEY"] = "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"
os.environ["PINECONE_API_KEY"] = "a7c62fa0-36d5-489c-83a1-198d568ce0be"

: 

In [5]:
import os

from groq import Groq

client = Groq(
    # This is the default and can be omitted
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "you are a helpful assistant."
        },
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

Fast language models, also known as rapid language models or quick language models, are a type of artificial intelligence (AI) system designed to process and generate human-like language at a remarkably high speed. They are a critical component of many natural language processing (NLP) applications, and their importance cannot be overstated. Here are some reasons why:

1. **Real-time processing**: Fast language models enable real-time processing of large amounts of language data, allowing for instant responses, recommendations, or completions. This is crucial for applications like chatbots, language translation, and text-based customer service.
2. **Scalability**: With the ability to process vast amounts of text data quickly, fast language models can handle massive volumes of user input, such as search queries, forum comments, or social media posts. This scalability is essential for large-scale NLP applications.
3. **Efficient inference**: Fast language models can perform inference (i.

In [7]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="75ee75fc-ed27-4292-8e81-56cd524452ec")

In [9]:
index_name = "quickstart"

pc.create_index(
    name=index_name,
    dimension=1024, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [10]:
data = [
    {"id": "vec1", "text": "Apple is a popular fruit known for its sweetness and crisp texture."},
    {"id": "vec2", "text": "The tech company Apple is known for its innovative products like the iPhone."},
    {"id": "vec3", "text": "Many people enjoy eating apples as a healthy snack."},
    {"id": "vec4", "text": "Apple Inc. has revolutionized the tech industry with its sleek designs and user-friendly interfaces."},
    {"id": "vec5", "text": "An apple a day keeps the doctor away, as the saying goes."},
    {"id": "vec6", "text": "Apple Computer Company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a partnership."}
]

embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[d['text'] for d in data],
    parameters={"input_type": "passage", "truncate": "END"}
)
print(embeddings[0])

{'values': [0.04925537109375, -0.01313018798828125, ..., -0.01971435546875, -0.01107025146484375]}


In [11]:
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

vectors = []
for d, e in zip(data, embeddings):
    vectors.append({
        "id": d['id'],
        "values": e['values'],
        "metadata": {'text': d['text']}
    })

index.upsert(
    vectors=vectors,
    namespace="ns1"
)

{'upserted_count': 6}

In [12]:
print(index.describe_index_stats())

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


In [13]:
query = "give me the detailed code with steps for creating a new index in pinecone"

embedding = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={
        "input_type": "query"
    }
)

In [14]:
results = index.query(
    namespace="ns1",
    vector=embedding[0].values,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [], 'namespace': 'ns1', 'usage': {'read_units': 1}}


In [15]:
for result in results['matches']:
    print(result['metadata']['text'])

In [16]:
import streamlit as st
import streamlit as st
import base64
from groq import Groq

client = Groq(
    # This is the default and can be omitted
    api_key=os.environ.get("GROQ_API_KEY"),
)

st.session_state.api_key = os.environ.get("GROQ_API_KEY")

# Only show the API key input if the key is not already set
if not st.session_state.api_key:
    # Ask the user's API key if it doesn't exist
    api_key = st.text_input("Enter API Key", type="password")
    
    # Store the API key in the session state once provided
    if api_key:
        st.session_state.api_key = api_key
        st.rerun()  # Refresh the app once the key is entered to remove the input field
else:
    # If the API key exists, show the chat app
    st.title("Chat App")

    # Initialize the chat message list in session state if it doesn't exist
    if "chat_messages" not in st.session_state:
        st.session_state.chat_messages = []

    # Display previous chat messages
    for messages in st.session_state.chat_messages:
        if messages["role"] in ["user", "assistant"]:
            with st.chat_message(messages["role"]):
                st.markdown(messages["content"])
    
    # Define a function to simulate chat interaction (you would replace this with an actual API call)
    def get_chat():
        embedding = pc.inference.embed(
            model="multilingual-e5-large",
            inputs=[st.session_state.chat_messages[-1]["content"]],
            parameters={
                "input_type": "query"
            }
        )
        results = index.query(
            namespace="ns1",
            vector=embedding[0].values,
            top_k=3,
            include_values=False,
            include_metadata=True
        )
        context = ""
        for result in results['matches']:
            context += result['metadata']['text'] + "\n"
            
        chat_completion = client.chat.completions.create(
            messages=st.session_state.chat_messages,
            model="llama3-8b-8192",
        )
        return chat_completion.choices[0].message.content

    # Handle user input
    if prompt := st.chat_input("Try asking the bot what it can do, or thank it for its help!"):
        # Display user message
        with st.chat_message("user"):
            st.markdown(prompt)
        st.session_state.chat_messages.append({"role": "user", "content": prompt})
        # Get the assistant's response (in this case, it's just echoing the prompt)
        with st.spinner("Getting responses..."):
            response = get_chat()
        with st.chat_message("assistant"):
            st.markdown(response)
        
        # Add user message and assistant response to chat history
        st.session_state.chat_messages.append({"role": "assistant", "content": response})

2024-10-17 15:58:47.695 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 15:58:47.699 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2024-10-17 15:58:47.701 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 15:58:47.703 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 15:58:47.704 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 15:58:48.750 
  command:

    streamlit run C:\Users\ANANTHA KRISHNA S

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load your dataset from the local directory
new_file_path = 'D:/CODE BOT 2.0/data set/Crop_recommendation.csv'
crop_data = pd.read_csv(new_file_path)

# Drop the 'label' column as it's the target
features = crop_data.drop(columns=['label']).values

# Normalize the features
scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(features)

# Now you have normalized data ready for embedding into Pinecone
print(normalized_features[:5])  # Display the first few normalized vectors for verification

#Re-load the dataset to ensure we're working with the latest upload
new_file_path = 'D:\CODE BOT 2.0\data set\Crop_recommendation.csv'
crop_data = pd.read_csv(new_file_path)

# Drop the label column as it's the target
features = crop_data.drop(columns=['label']).values

# Normalize the features for embedding into Pinecone
scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(features)

# Now we have the normalized data which is ready for embedding into Pinecone
normalized_features[:5]  # Display the first few normalized vectors for verification

: 